# 합성곱 신경망의 시각화

In [2]:
!curl -L https://bit.ly/3DQeEH8 -o best_cnn_model.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   129  100   129    0     0    492      0 --:--:-- --:--:-- --:--:--   498

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  1 1306k    1 21876    0     0  20313      0  0:01:05  0:00:01  0:01:04 20313
100 1306k  100 1306k    0     0  1182k      0  0:00:01  0:00:01 --:--:-- 46.4M
